__Finetuning torchvision models__

1. [Load tools](#Load-tools)
1. [Inputs](#Inputs)
1. [Helper functions](#Helper-functions)
    1. [Model training and validation code](#Model-training-and-validation-code)
    1. [Set model parameters’ .requires_grad attribute](#Set-model-parameters’-.requires_grad-attribute)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)


# Load tools

<a id = 'Load-tools'></a>

In [1]:
# Standard libary and settings
import os
import sys
import warnings; warnings.simplefilter('ignore')
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))

# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.6f}'.format

# import PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
import torch. autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.jit import script, trace
import torchvision
import torchvision.transforms as transforms

# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt

# Magic functions
%matplotlib inline


# Inputs

<a id = 'Inputs'></a>

In [ ]:
# image file directory that conforms to ImageFolder structure
data_dir = 'C:/Users/petersont/Desktop/data/hymenoptera_data'

# models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = 'squeezenet'

# number of classes to predict
num_classes = 2

# batch size
batch_size = 8

# number of epochs to train for
num_epochs = 15

# flag to indicate whether we are feature extracting or fine tuning
feature_extract = True


# Helper functions

<a id = 'Helper-functions'></a>

## Model training and validation code

<a id = 'Model-training-and-validation-code'></a>

In [ ]:
#
import copy
def train_model(model, dataloaders, criterion, optimizer, num_epochs = 25, is_inception = False):
    since = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {} / {}'.format(epoch, num_epochs - 1))
        print('-' * 25)
        
        # each epoch has a training and validation phase
        for phase in ['train','val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            running_loss= 0.0
            running_corrects = 0
            
            # iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the gradients
                optimizer.zero_grad()
                
                # forward pass
                # track history when in training mode
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        ouputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    
                    _, preds = torch.max(outputs, 1)
                                        
                    # backward pass in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                # capture running statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            else:
                val_acc_history.append(epoch_acc)
        
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val accuracy: {:.4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


## Set model parameters’ .requires_grad attribute

<a id = 'Set-model-parameters’-.requires_grad-attribute'></a>

In [1]:
#
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            

# A

<a id = ''></a>

# A

<a id = ''></a>